# Tests on Extreme Inputs

The following test section has the purpose is comprising tests that we expect to fail. This comprise tests that will show the limitation when working with floating point numbers. In particular this relates to testing for input values (x) in our matrices that exceed our limit of x < sqrt(2**53/n). This test does not include test on small numbers but we would expect same behavior with a set of very small numbers, as that ultimately also could result in rounding error. Namely numbers between 0.0-1.0 that include decimals beyond what we expect for floating point accuracy.

#### 1) Test on Large Numbers: x > sqrt(2**53/n)

All tests are run with: numpy.assert_equals() numpy.assert_allclose() https://numpy.org/doc/stable/reference/generated/numpy.allclose.html


In [2]:
import sys
import os
  
# getting the name of the directory
# where the this file is present.
current = os.path.abspath('')
# Getting the parent directory name
# where the current directory is present.
parent = os.path.dirname(current)
sys.path.append(parent)
from parameters import *

from experiments import *
from matrix_implementations import *
import random
import numpy as np
import csv
import logging

### Correct Input Generator (Following x < np.sqrt(2**(53)/n))

In [3]:
def get_input_range(n):
    lower_bound = 0
    upper_bound = round(np.sqrt(float(2**(53))/n))
    input_range = [lower_bound, upper_bound]
    return input_range


def generate_input(n: int) -> np.ndarray:
    list= []
    input_range = get_input_range(n)
    for i in range(0,n*n):
        l = random.randint(input_range[0],int(input_range[1]))
        list.append(float(l))
    return np.array(list).reshape(n,n)



### Too Large Inputs 


In [4]:
def too_large_input_range(n, power):
    lower_bound = 0
    upper_bound = round(np.sqrt(float(2**(power)/n)))
    input_range = [lower_bound, upper_bound]
    return input_range

def generate_too_large_input(n, power=70):
    list= []
    input_range = too_large_input_range(n, power)
    for i in range(0,n*n):
        
        l = random.randint(input_range[0],int(input_range[1]))
        list.append(float(l))
    return np.array(list).reshape(n,n)


### Test Matrix Generation

In [5]:
n = 64
A = generate_too_large_input(n)
B = generate_too_large_input(n)
C = generate_too_large_input(n)


In [6]:
A_within = generate_input(n)
B_within = generate_input(n)
type(A)

numpy.ndarray

# Quick tests to with some basic inputs

Further down there is full test method for extended tests: Go check them out!

In [7]:
np_res = np.matmul(A,B)
res = elementary_multiplication(Matrix(n,n, A), Matrix(n,n,B))

In [8]:
np_res_within = np.matmul(A_within,B_within)
res_within = elementary_multiplication(Matrix(n,n, A_within), Matrix(n,n,B_within))

In [9]:
#np.testing.assert_allclose(res.tolist(), np_res, 0.01, 0.05)


In [10]:
np.testing.assert_array_equal(res_within.tolist(),np_res_within)

In [11]:
#np.testing.assert_equal(res.tolist(), np_res)


In [12]:
def test(algorithm, A:np.ndarray, B:np.ndarray, n:int, C:Matrix=None):
    try:
        if C == None:
            np.testing.assert_equal(algorithm(Matrix(len(A),len(A), A), Matrix(len(B),len(B),B)).tolist(), np.matmul(A,B))
            print("Ok")
        else:
            np.testing.assert_equal(algorithm(Matrix(len(A),len(A), A), Matrix(len(B),len(B),B), C).tolist(), np.matmul(A,B))
            print("Ok")
    except AssertionError as error:
        
        # Uncomment the log to see full error message each time
        logging.error("TEXT", exc_info=True)
        error_message = str(error)
        error_percentage = float(error_message.split("Mismatched elements: ")[1].split("%)")[0].split("(")[1])/100
        
        return error_percentage

#test(elementary_multiplication, A, B, n)

In [13]:
def full_test(algorithm_to_be_tested, input_generator, n:int, iterations:int, inplace=False, power:int=None):
    
    """full_test takes an algorithm that is to be tested and the specified input generator that creates the inputs for 
    the given test. It evaluates whether a power is specified. Here a power refers to the power to which the input is
    scaled with. it also checks if the algorithm being tested is an inplace algorithm or copying variant.
    
    """
    errors = []
    
    for i in range(iterations):
        
        if power != None:
            A = input_generator(n, power)
            B = input_generator(n, power)
        else: 
            A = input_generator(n)
            B = input_generator(n)
            
        if inplace == True:     
            val = test(algorithm_to_be_tested, A, B, C=Matrix(len(A),len(A)), n=n)
            if val != None:
                errors.append(val)
        else:
            val = test(algorithm_to_be_tested, A, B, n=n)
            if val != None:
                errors.append(val)
    
    return errors

### The Real Testing - Try out the methods and run some tests

The tests below uses the methods provided above to test on very large inputs. Here we in particular reveal how strassen's algorithm with the more arithmetic operations 

In [14]:
#Basic test on the recursive write through:
#writes ok for each test and returns an empty list if there are no mistakes in the test
full_test(recursive_multiplication_write_through, generate_input, n, iterations=3, inplace=True)


Ok
Ok
Ok


[]

### Strassen Failing on large inputs earlier than other methods


In [15]:
powers = [47,50,53,56,59,62,65,68,71,74,77]
res = {}

print("Please don't mind the Error log text. It can be # commented out in the test method if desired.")

for power in powers:
    temp = full_test(strassen, generate_too_large_input,n, iterations=3, inplace=False, power=power)
    total = 0
    if len(temp) != 0:
        for x in temp:
            total += x
        average = total/len(temp)
        res[power] = average
    else:
        res[power] = "No failed computations"


Please don't mind the Error log text. It can be # commented out in the test method if desired.
Ok
Ok
Ok


ERROR:root:TEXT
Traceback (most recent call last):
  File "/tmp/ipykernel_12857/2947564240.py", line 4, in test
    np.testing.assert_equal(algorithm(Matrix(len(A),len(A), A), Matrix(len(B),len(B),B)).tolist(), np.matmul(A,B))
  File "/home/katarzyna/.local/lib/python3.7/site-packages/numpy/testing/_private/utils.py", line 345, in assert_equal
    return assert_array_equal(actual, desired, err_msg, verbose)
  File "/home/katarzyna/.local/lib/python3.7/site-packages/numpy/testing/_private/utils.py", line 935, in assert_array_equal
    verbose=verbose, header='Arrays are not equal')
  File "/home/katarzyna/.local/lib/python3.7/site-packages/numpy/testing/_private/utils.py", line 844, in assert_array_compare
    raise AssertionError(msg)
AssertionError: 
Arrays are not equal

Mismatched elements: 156 / 4096 (3.81%)
Max absolute difference: 2.
Max relative difference: 8.36240556e-15
 x: array([[2.892727e+14, 3.322558e+14, 2.877808e+14, ..., 2.842317e+14,
        3.312153e+14, 3.497291e+14]

## Test result - Strassen on Large Numbers:
We thus see that strassen fails earlier than originally anticipated as it fails before the 2^53. The results can be inspected below in the res dictionary. The key defined the power where the power is 2^power. We expect the floats to work until the power of 53.

In [16]:
res

{47: 'No failed computations',
 50: 0.0257,
 53: 0.6070000000000001,
 56: 0.9096666666666667,
 59: 0.9159999999999999,
 62: 0.9203333333333333,
 65: 0.9273333333333333,
 68: 0.9263333333333333,
 71: 0.9233333333333333,
 74: 0.9203333333333333,
 77: 0.9249999999999999}